<a href="https://colab.research.google.com/github/rohithmsr/AI-practice/blob/main/Opik/1_multi_step_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Tracking a Multi-step LLM Chain

In this exercise, you'll track a multi-step LLM chain with Opik. You can use OpenAI or open source models via LiteLLM.

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If OpenAI is called within one of these steps, the LLM call with be associated with that corresponding step:

# Imports & Configuration

In [1]:
%pip install opik openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.0/418.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 3.6 MB/s eta 0:00:00


In [2]:
from opik import track
import opik
from opik.integrations.openai import track_openai
from openai import OpenAI
import getpass
import os

os.environ["OPIK_PROJECT_NAME"] = "Multi-step-Chain-Demo"

In [4]:
# opik configs
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

Enter your Opik API key: ··········
Do you want to use "rohithmsr" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


In [3]:
# openai configs
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
client = OpenAI()
openai_client = track_openai(client)

Enter your OpenAI API key: ··········


# Define First Step

In [5]:
@track
def generate_meal(ingredient):
    prompt = f"Generate one example of a meal that can be made with {ingredient}."
    res = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Define Second Step

In [6]:
@track
def generate_recipe(meal):
    prompt = f"Generate a step-by-step recipe for {meal}"
    res = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Call Chain

In [9]:
@track
def generate_recipe_from_ingredient(ingredient):
    meal = generate_meal(ingredient)
    story = generate_recipe(meal)
    return story

generate_recipe_from_ingredient("tender coconut")

'Ingredients:\n- 1 cup of white rice\n- 1 cup of coconut milk\n- 1 cup of water\n- 1/2 cup of tender coconut pieces\n- 1/4 cup of shredded coconut\n- Salt and pepper to taste\n- 4 fish fillets\n- 1/2 cup of coconut milk\n- 1 tablespoon of lime juice\n- 1 tablespoon of soy sauce\n- 1 tablespoon of honey\n- 1 teaspoon of minced garlic\n- 1 teaspoon of minced ginger\n- 1 tablespoon of vegetable oil\n\nInstructions:\n\n1. In a saucepan, combine the rice, coconut milk, water, tender coconut pieces, shredded coconut, salt, and pepper. Bring to a boil, then reduce heat to low and let simmer for about 20 minutes or until the rice is cooked and the liquid is absorbed.\n\n2. In a separate bowl, mix together the coconut milk, lime juice, soy sauce, honey, garlic, and ginger to make the coconut milk sauce. Set aside.\n\n3. Season the fish fillets with salt and pepper. Heat the vegetable oil in a grill pan over medium heat. Grill the fish fillets for about 3-4 minutes on each side, or until cooked 

# Try with your own example!

In [10]:
generate_recipe_from_ingredient(input("Enter an ingredient: "))

Enter an ingredient: Horsegram


'\n11. You can also add a squeeze of lemon juice before serving for an extra burst of flavor.\n12. This dish can be made vegan by using vegetable oil instead of ghee.\n13. Feel free to add in other vegetables like carrots, peas, or bell peppers for added nutrition and texture. \n14. Leftovers can be stored in an airtight container in the refrigerator for up to 3-4 days. Reheat before serving. Enjoy!'